In [2]:
import torch
from torch import nn
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 汇聚层

- 当处理图像时，希望逐渐降低隐藏表示的空间分辨率、聚集信息
- 这样随着在神经网络中层叠的上升，每个神经元对其敏感的感受野（输入）就越大

\begin{definition}\label{def:receptiveField}
感受野（receptive field）：在深度学习网络中，某个层的神经元的感受野指的是在前向传播期间可能影响该神经元计算的来自所有先前层的所有元素。
\end{definition}


![](../img/6_convolutional_neural_networks/receptiveField.gif)

- 当特征图中的任何元素需要更大的感受野来检测更宽区域的输入特征时，可以构建更深的网络

- 机器学习任务通常会跟全局图像的问题有关，所以最后一层的神经元应该对整个输入的全局敏感
- 通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层

- 卷积对位置敏感
    - 例如，由于照明、物体位置偏离、外观等因素造成物体在图像中的位置移动

例如，检测物体的边缘（红色边）

![](../img/6_convolutional_neural_networks/kernelSensitive.svg)

红色边向右移动一个像素，特征映射中的位置也相应改变

因此，用上半部分训练好的模型就不能够识别出下半部分移位后物体的红边

- 需要一定程度的平移不变性
    - 将输入数据移动小距离，特征映射的大部分结果应当不变
    - 特别是当只关心物体是否在图像中，而不关心它的具体位置的时候

\begin{definition}\label{def:pooling}
**汇聚（池化）（pooling）**：将特征映射的数值用该特征映射位置周边数值的统计量替换
\end{definition}


- 汇聚层具有双重目的：
    - 降低卷积层对位置的敏感性，
    - 同时降低对空间降采样表示的敏感性

## 最大汇聚层和平均汇聚层

- 汇聚与卷积有类似之处，也是有一个滑动窗口扫过输入数据来计算输出，但是没有核和点积操作
- 滑动窗口：固定形状，有时被称为“汇聚窗口”

- 汇聚层不包含参数，运算是确定的

\begin{definition}\label{def:maxPooling}
**最大汇聚层（maximum pooling）：返回汇聚窗口中所有元素的最大值
\end{definition}


\begin{definition}\label{def:avePooling}
**平均汇聚层（average pooling）：返回汇聚窗口中所有元素的平均值
\end{definition}


\begin{example}\label{example:maxPooling}
汇聚窗口形状为 $2\times 2$ 的最大汇聚层
\end{example}


![](pooling1.png)

\begin{definition}\label{def:pqPooling}
**$p \times q$汇聚层**：汇聚窗口形状为$p \times q$的汇聚层。汇聚操作称为$p \times q$汇聚。
\end{definition}


![](pooling2.png)

- 如果原始图片有1像素移位，经历汇聚层后，依然可以保留检测边缘的能力。
- 有模糊化的效果。

- 构建汇聚层函数，实现汇聚层前向传播

In [4]:
def pool2d(X, pool_size, mode='max'): # 单通道，没有padding、stride
    p_h, p_w = pool_size  # 汇聚窗口的高、宽
    # 初始化输出Y
    Y = torch.zeros((X.shape[0]-p_h+1, X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h, j:j+p_w].max()
            if mode == 'avg':
                Y[i,j] = X[i:i+p_h, j:j+p_w].mean()
    return Y

验证二维最大汇聚层的输出

In [6]:
X = torch.tensor([[0.0,1.0,2.0], [3.0,4.0,5.0], [6.0,7.0,8.0]])
print(f'X {X}')
print(f'汇聚输出\n{pool2d(X,(2,2))}')

X tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

汇聚输出
tensor([[4., 5.],
        [7., 8.]])

验证平均汇聚层

In [8]:
print(f'X {X}')
print(f"平均汇聚输出\n{pool2d(X,(2,2),'avg')}")

X tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

平均汇聚输出
tensor([[2., 3.],
        [5., 6.]])

## 填充和步幅

- 汇聚层与卷积层类似，都具有填充和步幅

\begin{example}\label{example:torchPooling}
用深度学习框架中内置的二维最大汇聚层，来演示汇聚层中填充和步幅的使用
\end{example}


### 默认情形

第一步：调整输入的形状

In [9]:
# 样本数是1，输入通道数是1，形状4*4
X = torch.arange(16, dtype=torch.float32 ).reshape((1,1,4,4)) 
X


tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

第二步：调用nn.MaxPool2d

In [10]:
pool2d = nn.MaxPool2d(3) # 3是窗口的高宽
pool2d(X)

tensor([[[[10.]]]])

>【注意】深度学习框架中默认的汇聚的步幅与汇聚窗口的大小相同

### 手动设定填充和步幅

(1)一般情形

In [11]:
pool2d = nn.MaxPool2d(3,padding=1,stride=2)

print(f'X {X}')
print(f'汇聚输出\n{pool2d(X)}')

X tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

汇聚输出
tensor([[[[ 5.,  7.],
          [13., 15.]]]])

(2)设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度

In [12]:
pool2d = nn.MaxPool2d((2,3),padding=(1,1),stride=(2,3))
print(f'X {X}')
print(f'汇聚输出\n{pool2d(X)}')

X tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

汇聚输出
tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

## 多个通道

- 在**每个输入通道**应用池化层以获得**相应的**输出通道（不像卷积核，会融合多个输入通道）
- 输出通道数<mark>$\bf{=}$</mark>输入通道数

\begin{example}\label{example:multiChannels}
在通道维度上连结张量$X$和$X + 1$，以构建具有2个通道的输入
\end{example}


In [17]:
print(f'张量 X {X}\n维度为 {X.shape}')

张量 X tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
维度为 torch.Size([1, 1, 4, 4])

In [18]:
X = torch.cat((X,X + 1), 1)
print(f'新张量X {X}\n维度为{X.shape}')

新张量X tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])
维度为torch.Size([1, 2, 4, 4])

In [19]:
pool2d = nn.MaxPool2d(3,padding=1,stride=2)
# 汇聚后输出通道数仍为2
pool2d(X)


tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])